# Get Dataset from Kaggle

In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
print("done")
! kaggle datasets download -d tomasslama/indoor-climbing-gym-hold-segmentation
! unzip indoor-climbing-gym-hold-segmentation.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
done
Dataset URL: https://www.kaggle.com/datasets/tomasslama/indoor-climbing-gym-hold-segmentation
License(s): CC-BY-SA-4.0
100% 11.1G/11.1G [02:00<00:00, 106MB/s] 
100% 11.1G/11.1G [02:00<00:00, 99.1MB/s]
Archive:  indoor-climbing-gym-hold-segmentation.zip
  inflating: bh-annotation.csv       
  inflating: bh-annotation.json      
  inflating: bh-phone-annotation.csv  
  inflating: bh-phone-annotation.json  
  inflating: bh-phone/000.jpg        
  inflating: bh-phone/001.jpg        
  inflating: bh-phone/002.jpg        
  inflating: bh-phone/003.jpg        
  inflating: bh-phone/004.jpg        
  inflating: bh-phone/005.jpg        
  inflating: bh-phone/006.jpg        
  inflating: bh-phone/007.jpg        
  inflating: bh-phone/008.jpg        
  inflating: bh-phone/009.jpg        
  inflating: bh-phone/010.jpg        
  inflating: bh-phone/011.jpg       

# ML

Installing yolov5

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

%pip install -q clearml
import clearml; clearml.browser_login()

YOLOv5 🚀 v7.0-318-gc0380fd8 Python-3.10.12 torch-2.3.0+cu121 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 52.5/107.7 GB disk)


<IPython.core.display.Javascript object>

ClearML automatic browser login failed, please login or create a new account
To get started with ClearML: setup your own `clearml-server`, or create a free account at https://app.clear.ml

Please login to https://app.clear.ml , then press [Enter] to connect 


<IPython.core.display.Javascript object>


We cannot connect automatically (adblocker / incognito?) 😟 
Please go to https://app.clear.ml/settings/workspace-configuration 
Then press  + Create new credentials  
And copy/paste your Access Key here: OI20VMNBC7P5OCQU1QRL
 Setting access key 
Now copy/paste your Secret Key here: xAUt3mAWuj2JzEuNlQEKrLf3WgDfQYigBZy0VpFBEPbZsXtJkZ
 Setting secret key 


Hurrah! 🥳 🎊 🎉
🤖 ClearML connected successfully - let's build something! 🚀


Creating clearML task

In [ ]:
from clearml import Task
task = Task.init(project_name="HoldsDetection", task_name="firstTask")

ClearML Task: created new task id=6a5694bdd92645b6ba0f1a2e79fedc2d
ClearML results page: https://app.clear.ml/projects/cc259027f84d4ab0981ef2a601bdff7e/experiments/6a5694bdd92645b6ba0f1a2e79fedc2d/output/log


/usr/local/lib/python3.10/dist-packages/clearml/utilities/process/mp.py:622: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



## Setup Dataset

utils:

In [3]:
import json
import os
import torch
import torchvision


def convert_json_to_yolo(json_file, labels_dir):
  # labels_dir = '/content/sm/labels'
  os.makedirs(labels_dir , exist_ok=True)

  with open(json_file) as f:
    data = json.load(f)

    for key in data:
      image = data[key]
      filename = image['filename']
      regions = image['regions']

      if not regions:
        continue

      txt_filename = os.path.splitext(filename)[0] + '.txt'
      txt_filepath = os.path.join(labels_dir, txt_filename)

      with open(txt_filepath, 'w') as txt_file:
        for region in regions:
          shape_attributes = region['shape_attributes']
          region_attributes = region['region_attributes']
          class_name = region_attributes['hold_type']

          all_points_x = shape_attributes['all_points_x']
          all_points_y = shape_attributes['all_points_y']

          width = max(all_points_x) - min(all_points_x)
          height = max(all_points_y) - min(all_points_y)
          center_x = min(all_points_x) + width / 2
          center_y = min(all_points_y) + height / 2

          label = f'{class_name} {center_x} {center_y} {width} {height}\n'
    txt_file.write(label)

def split_train_val_test(json_file):
    images_dir = '/content/sm/images'
    labels_dir = '/content/sm/labels'
    train_ratio = 0.8
    test_ratio = 0.2


    torch.random.manual_seed(42)
    files = os.listdir(images_dir)

    train_size = int(train_ratio * len(files))
    test_size = int(len(files) - train_size)


    train_choose = torch.randperm(len(files))[:train_size]
    training_files = [files[i] for i in train_choose]
    test_files = [file for file in files if file not in training_files]


In [ ]:
!rm /content/bh/labels/*
